# RAG 사용 유무에 따른 피드백 결과 비교

In [15]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage(model="solar-pro")
llm.invoke("Hello, world!")

AIMessage(content="Hello, world! It's great to meet you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 14, 'total_tokens': 34, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'solar-pro-241126', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-682cce81-9f5e-4725-8460-672b9daf9774-0', usage_metadata={'input_tokens': 14, 'output_tokens': 20, 'total_tokens': 34, 'input_token_details': {}, 'output_token_details': {}})

## 1. 피드백

In [16]:
import time

In [25]:
# 예시 텍스트
text = """
별의 형성 성운 단계:성운(가스와 먼지 구름)이 온도 하강으로 인해 수축이 시작.중심부가 차가워지며 밀도가 높아지고, 원시별이 형성됨. 
원시별 단계:핵융합 반응이 바로 시작되며 중심부 온도가 약 1천만 K로 상승. 
주계열성 단계 핵융합 시작:중심부 온도가 약 10억 K에 도달하면 수소가 헬륨으로 융합되며 에너지를 방출. 
특징:별은 내부의 중력과 외부로 나가는 방사압이 균형을 이루지 못하며 진동을 반복.질량이 클수록 수명이 길어지고 더 밝으며 온도는 낮아짐. 
거성 또는 초거성 단계 핵융합 종료 후 부풀어오름:중심부의 수소가 모두 소모되면 별 전체가 서서히 수축하며 적색 거성 또는 초거성이 됨. 
핵융합 반복:질량이 큰 별은 헬륨 이후에도 탄소, 산소, 규소 등을 융합하며 금까지 생성. 별의 최후: 질량에 따른 진화
(1) 질량이 작은 별 (태양과 비슷한 별) 행성상 성운:핵융합 종료 후 바깥층이 우주로 방출되지 않고 중심부로 흡수됨. 백색왜성:중심부가 폭발하여 작은 규모의 초신성이 형성됨.
(2) 질량이 큰 별 초신성 폭발:핵융합으로 헬륨까지만 생성된 후 더 이상 에너지를 방출할 수 없어 중력이 붕괴. 중성자별:초신성 폭발 후 남은 별의 중심부가 수소로 이루어진 별로 변환됨. 블랙홀:질량이 큰 별은 외곽층이 모두 날아가며, 작은 블랙홀이 형성됨. 별의 진화 요약 질량이 작은 별 → 초거성 → 행성상 성운 → 백색왜성 질량이 큰 별 → 적색 거성 → 초신성 → (질량에 따라) 행성상 성운 또는 중성자별
"""

In [26]:
def provide_feedback(input_text):
    # LLM을 이용해 피드백 요청
    llm = ChatUpstage(api_key=UPSTAGE_API_KEY,model="solar-pro")
    
    prompt = f"""
    다음 사용자가 공부하며 작성한 텍스트에 대해 수능 문제를 푸는 사람을 대상으로 피드백을 작성해주세요.
    잘못된 개념 학습을 방지하기 위해 피드백을 제공하고자 합니다.
    피드백은 틀린 내용, 고칠 내용, 해설 순으로 작성되어야 하며, 피드백이 여러 개 있을 경우 번호를 매기세요.
    해설은 간결하게 제공해주세요.
    텍스트: {input_text}
    """

    # LLM 호출
    response = llm.invoke(prompt)
    
    # 전체 응답을 그대로 반환
    return response.content

start = time.time()
# 피드백 제공
feedback = provide_feedback(text)

# 전체 피드백 출력
print(feedback)

end = time.time()
print(end-start)

1. 고칠 내용: "별은 내부의 중력과 외부로 나가는 방사압이 균형을 이루지 못하며 진동을 반복." 이 문장은 틀렸습니다. 별은 내부의 중력과 외부로 나가는 방사압이 균형을 이루며 안정적으로 존재합니다.

   해설: 별은 내부에서 일어나는 핵융합 반응으로 인해 생성된 에너지가 외부로 방출되고, 이 방사압이 중력과 균형을 이루어 안정적으로 존재하는 것입니다.

2. 고칠 내용: "질량이 클수록 수명이 길어지고 더 밝으며 온도는 낮아짐." 이 문장은 틀렸습니다. 질량이 클수록 수명이 짧아지고 더 밝으며 온도는 높아집니다.

   해설: 질량이 큰 별은 더 많은 연료를 빠르게 소모하기 때문에 수명이 짧습니다. 또한, 더 많은 에너지를 생성하여 더 밝으며, 높은 온도를 유지합니다.

3. 고칠 내용: "질량이 작은 별 (태양과 비슷한 별) 행성상 성운:핵융합 종료 후 바깥층이 우주로 방출되지 않고 중심부로 흡수됨." 이 문장은 틀렸습니다. 질량이 작은 별은 핵융합 종료 후 바깥층이 우주로 방출되어 행성상 성운을 형성합니다.

   해설: 질량이 작은 별은 핵융합 종료 후 외부층이 팽창하여 행성상 성운을 형성하고, 중심부는 백색왜성으로 남게 됩니다.

4. 고칠 내용: "중심부가 폭발하여 작은 규모의 초신성이 형성됨." 이 문장은 틀렸습니다. 백색왜성은 중심부가 폭발하지 않고, 핵융합 연료가 모두 소모되어 남은 별의 잔해입니다.

   해설: 백색왜성은 핵융합 연료가 모두 소모된 후 남은 별의 잔해로, 폭발하지 않습니다.

5. 고칠 내용: "질량이 큰 별 초신성 폭발:핵융합으로 헬륨까지만 생성된 후 더 이상 에너지를 방출할 수 없어 중력이 붕괴." 이 문장은 틀렸습니다. 질량이 큰 별은 헬륨 이후에도 더 많은 원소를 융합하며, 핵융합이 종료되면 중력이 붕괴하여 초신성 폭발을 일으킵니다.

   해설: 질량이 큰 별은 헬륨 이후에도 탄소, 산소, 규소 등을 융합하며, 핵융합이 종료되면 중력이 붕괴하여 초신성 폭발을 일으킵니다.

6. 고칠 내용: "중성자별:초신성 폭

In [27]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_upstage import ChatUpstage, UpstageEmbeddings
from pinecone import Pinecone, ServerlessSpec

In [28]:
import os

def langsmith(project_name=None, set_enable=True):

    if set_enable:
        result = os.environ.get("LANGCHAIN_API_KEY")
        if result is None or result.strip() == "":
            print("LangChain API Key가 설정되지 않았습니다. 참고: https://wikidocs.net/250954")
            return
        os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"  # LangSmith API 엔드포인트
        os.environ["LANGCHAIN_TRACING_V2"] = "true"  # true: 활성화
        os.environ["LANGCHAIN_PROJECT"] = project_name  # 프로젝트명
        print(f"LangSmith 추적을 시작합니다.\n[프로젝트명]\n{project_name}")
    else:
        os.environ["LANGCHAIN_TRACING_V2"] = "false"  # false: 비활성화
        print("LangSmith 추적을 하지 않습니다.")


In [29]:
def analysis_chunk(input_data):
    # LangSmith 시작
    langsmith("rag-test")
    db_index_name = "dev-02"

    # Pinecone 연결
    pc = Pinecone()
    index = pc.Index(db_index_name)
    embeddings = UpstageEmbeddings(model="embedding-query")
    vectorstore = PineconeVectorStore(index=index, embedding=embeddings)

    # 검색기(Retriever) 생성
    retriever = vectorstore.as_retriever()

    # 입력 데이터와 관련된 정보 검색
    retrieved_docs = retriever.invoke(input_data)
    doc = retrieved_docs[0]
    doc_context = doc.page_content
    rag_id = doc.id

    # 프롬프트 템플릿 생성 (피드백 형식)
    prompt = PromptTemplate.from_template(
        """
        다음 사용자가 공부하며 작성한 텍스트에 대해 수능 문제를 푸는 사람을 대상으로 피드백을 작성해주세요.
        잘못된 개념 학습을 방지하기 위해 피드백을 제공하고자 합니다.
        피드백은 틀린 내용, 고칠 내용, 해설 순으로 작성되어야 하며, 피드백이 여러 개 있을 경우 번호를 매기세요.
        해설은 간결하게 제공해주세요.
    

        #정보:
        {context}

        #입력:
        {question}

        #답:"""
    )
    
    # 언어모델(LLM) 생성
    llm = ChatUpstage(model="solar-pro")
    
    # RAG: 문서 검색 후 피드백 생성
    chain = RunnablePassthrough() | prompt | llm | StrOutputParser()
    
    # RAG 체인 실행
    response = chain.invoke({"context": doc_context, "question": input_data})

    # 결과 반환
    return {
        "rag_id": rag_id,
        "response": response,
    }

start = time.time()
# 피드백 제공
feedback = analysis_chunk(text)

# 전체 피드백 출력
print(feedback['response'])
end = time.time()
print(f'time: {end-start}')

LangSmith 추적을 시작합니다.
[프로젝트명]
rag-test
1. 잘못된 내용: "질량이 작은 별 (태양과 비슷한 별) 행성상 성운:핵융합 종료 후 바깥층이 우주로 방출되지 않고 중심부로 흡수됨. 백색왜성:중심부가 폭발하여 작은 규모의 초신성이 형성됨."

   고칠 내용: "질량이 작은 별 (태양과 비슷한 별) 행성상 성운:핵융합 종료 후 중심핵이 붕괴하면서 바깥층이 우주로 방출되어 행성상 성운을 형성. 백색왜성:중심부가 붕괴하고 폭발하지 않아 작은 크기의 백색왜성이 형성됨."

   해설: 행성상 성운은 핵융합 종료 후 중심핵이 붕괴하면서 형성된 것이며, 백색왜성은 중심부가 폭발하지 않고 남은 잔해입니다.

2. 잘못된 내용: "질량이 큰 별 초신성 폭발:핵융합으로 헬륨까지만 생성된 후 더 이상 에너지를 방출할 수 없어 중력이 붕괴. 중성자별:초신성 폭발 후 남은 별의 중심부가 수소로 이루어진 별로 변환됨. 블랙홀:질량이 큰 별은 외곽층이 모두 날아가며, 작은 블랙홀이 형성됨."

   고칠 내용: "질량이 큰 별 초신성 폭발:핵융합으로 헬륨 이후에도 더 많은 원소를 생성하다가, 더 이상 에너지를 방출할 수 없어 중력이 붕괴. 중성자별:초신성 폭발 후 남은 별의 중심부가 중성자로 이루어진 밀집한 별로 변환됨. 블랙홀:질량이 매우 큰 별은 외곽층이 모두 날아가며, 강한 중력으로 인해 빛조차 탈출할 수 없는 블랙홀이 형성됨."

   해설: 중성자별은 중성자로 이루어진 밀집한 별이며, 블랙홀은 질량이 매우 큰 별의 중력 붕괴로 형성됩니다.
time: 12.439532041549683


## 2. 퀴즈

In [30]:
def provide_feedback(input_text):
    # LLM을 이용해 피드백 요청
    llm = ChatUpstage(api_key=UPSTAGE_API_KEY,model="solar-pro")
    
    prompt = f"""
    아래 텍스트를 기반으로 수능 수준의 O/X 퀴즈 5개를 만들어주세요. 
    다음 조건을 충족해야 합니다:
    1. 질문은 개념 이해와 문제 해결 능력을 평가할 수 있어야 합니다.
    2. 정답이 "O"인 문제와 "X"인 문제의 비율은 균형 있게 구성해주세요.
    3. 질문의 난이도는 수능 수준에 맞춰 구체적이고 사고를 요하는 내용을 포함해야 합니다.
    4. 각 질문의 정답에 대한 설명은 간결하지만 충분히 납득 가능하게 작성해주세요.
    
    텍스트: {input_text}
    """

    # LLM 호출
    response = llm.invoke(prompt)
    
    # 전체 응답을 그대로 반환
    return response.content

start = time.time()
# 피드백 제공
feedback = provide_feedback(text)

# 전체 피드백 출력
print(feedback)

end = time.time()
print(end-start)

1. 별의 형성 과정에서 성운 단계에서는 중력이 증가하여 중심부가 수축하기 시작한다. (O/X)
   정답: O
   설명: 성운 단계에서 가스와 먼지 구름의 온도 하강으로 인해 수축이 시작되고, 중심부의 밀도가 높아진다.

2. 원시별 단계에서 중심부의 온도는 약 1천만 K까지 상승한다. (O/X)
   정답: X
   설명: 원시별 단계에서 중심부의 온도는 약 1천만 K까지 상승하지만, 핵융합 반응이 바로 시작되지는 않는다.

3. 별이 진화하면서 주계열성 단계에서 내부의 중력과 외부로 나가는 방사압이 균형을 이루지 못하면 진동을 반복한다. (O/X)
   정답: O
   설명: 주계열성 단계에서 별은 내부와 외부의 힘의 균형을 이루지 못하며 진동을 반복한다.

4. 질량이 작은 별은 적색 거성 단계를 거치지 않고 바로 백색왜성이 된다. (O/X)
   정답: X
   설명: 질량이 작은 별(태양과 비슷한 별)은 적색 거성 단계를 거친 후 행성상 성운과 백색왜성 단계를 거친다.

5. 질량이 큰 별은 초신성 폭발 후 중성자별이 될 수 있다. (O/X)
   정답: O
   설명: 질량이 큰 별은 초신성 폭발 후 남은 별의 중심부가 수소로 이루어진 별로 변환되어 중성자별이 될 수 있다.
7.7527031898498535


In [31]:
def analysis_chunk(input_data):
    # LangSmith 시작
    langsmith("rag-test")
    db_index_name = "dev-02"

    # Pinecone 연결
    pc = Pinecone()
    index = pc.Index(db_index_name)
    embeddings = UpstageEmbeddings(model="embedding-query")
    vectorstore = PineconeVectorStore(index=index, embedding=embeddings)

    # 검색기(Retriever) 생성
    retriever = vectorstore.as_retriever()

    # 입력 데이터와 관련된 정보 검색
    retrieved_docs = retriever.invoke(input_data)
    doc = retrieved_docs[0]
    doc_context = doc.page_content
    rag_id = doc.id

    # 프롬프트 템플릿 생성 (피드백 형식)
    prompt = PromptTemplate.from_template(
        """
        아래 텍스트를 기반으로 수능 수준의 O/X 퀴즈 5개를 만들어주세요. 
        다음 조건을 충족해야 합니다:
        1. 질문은 개념 이해와 문제 해결 능력을 평가할 수 있어야 합니다.
        2. 정답이 "O"인 문제와 "X"인 문제의 비율은 균형 있게 구성해주세요.
        3. 질문의 난이도는 수능 수준에 맞춰 구체적이고 사고를 요하는 내용을 포함해야 합니다.
        4. 각 질문의 정답에 대한 설명은 간결하지만 충분히 납득 가능하게 작성해주세요.
    

        #정보:
        {context}

        #입력:
        {question}

        #답:"""
    )
    
    # 언어모델(LLM) 생성
    llm = ChatUpstage(model="solar-pro")
    
    # RAG: 문서 검색 후 피드백 생성
    chain = RunnablePassthrough() | prompt | llm | StrOutputParser()
    
    # RAG 체인 실행
    response = chain.invoke({"context": doc_context, "question": input_data})

    # 결과 반환
    return {
        "rag_id": rag_id,
        "response": response,
    }

start = time.time()
# 피드백 제공
feedback = analysis_chunk(text)

# 전체 피드백 출력
print(feedback['response'])
end = time.time()
print(f'time: {end-start}')

LangSmith 추적을 시작합니다.
[프로젝트명]
rag-test
1. 태양보다 질량이 작은 별은 적색 거성이 된 후, 초신성 폭발을 일으킨다. (X)
   설명: 태양보다 질량이 작은 별은 적색 거성이 된 후, 초신성 폭발을 일으키지 않고, 행성상 성운 단계를 거쳐 백색 왜성으로 진화한다.

2. H-R도에서 가로축은 별의 표면 온도를 나타내고, 세로축은 절대 등급을 나타낸다. (O)
   설명: H-R도는 별의 표면 온도와 절대 등급을 나타내는 도표이다.

3. 원시별 단계에서 중심부의 온도는 약 1천만 K이다. (O)
   설명: 원시별 단계에서 중심부의 온도는 약 1천만 K로 상승하며, 핵융합 반응이 시작된다.

4. 주계열성 단계에서 별은 내부의 중력과 외부로 나가는 방사압이 균형을 이룬다. (O)
   설명: 주계열성 단계에서 별은 내부의 중력과 외부로 나가는 방사압이 균형을 이루며, 진동을 반복한다.

5. 질량이 큰 별은 초신성 폭발 후 남은 별의 중심부가 블랙홀로 진화한다. (O)
   설명: 질량이 큰 별은 초신성 폭발 후 남은 별의 중심부가 수소로 이루어진 별로 변환되며, 블랙홀이 형성될 수 있다.
time: 11.519300937652588
